In [1]:
import pandas as pd

from code.parser.parse import Parser
import matplotlib.pyplot as plt
from code.wiener_kolmogorov.wiener_kolmogorov import WienerKolmogorov as WK
from code.detector.detector import  Detector
import numpy as np
import datetime

pre1 = Parser.read('../../runs/1/pre.txt')
abs1 = Parser.read('../../runs/1/abs.txt')
pre2 = Parser.read('../../runs/2/pre.txt')
abs2 = Parser.read('../../runs/2/abs.txt')

In [2]:
pre1 = pd.concat([pre1, abs1], ignore_index=True)
abs1 = pd.concat([abs1, abs1], ignore_index=True)
pre2 = pd.concat([pre2, abs2], ignore_index=True)
abs2 = pd.concat([abs2, abs2], ignore_index=True)

In [3]:
pre1 = pd.concat([pre1, pre2], ignore_index=True)

In [7]:
pre1.rssi.to_numpy()

array([-29., -27., -26., -27., -26., -26., -26., -27., -27., -27., -27.,
       -27., -32., -35., -34., -38., -28., -38., -27., -38., -38., -34.,
       -38., -38., -38., -35., -27., -38., -38., -38., -27., -38., -38.,
       -35., -38., -38., -38., -35., -38., -32., -38., -35., -27., -38.,
       -38., -38., -38., -38., -35., -38., -20., -20., -19., -20., -21.,
       -19., -21., -18., -22., -20., -20., -20., -19., -22., -19., -18.,
       -22., -23., -18., -19., -21., -21., -20., -21., -18., -21., -22.,
       -22., -20., -21., -24., -20., -23., -20., -20., -22., -24., -19.,
       -20., -17., -19., -18., -20., -20., -19., -21., -22., -20., -20.,
       -19., -20., -20., -19., -20., -21., -19., -21., -18., -22., -20.,
       -20., -20., -19., -22., -19., -18., -22., -23., -18., -19., -21.,
       -21., -20., -21., -18., -21., -22., -22., -20., -21., -24., -20.,
       -23., -20., -20., -22., -24., -19., -20., -17., -19., -18., -20.,
       -20., -19., -21., -22., -20., -20., -19., -1

In [4]:
y2 = np.concatenate((np.ones(50), np.zeros(150)))

In [5]:
y1 = np.concatenate([y2, y2])

In [6]:
from sklearn.metrics import accuracy_score

In [16]:
def filter_out(pres, abss):
    filters = []
    for abs in abss:
        filter = WK()
        filter.fit(abs["rssi"].to_numpy())
        filters.append(filter)
    outs = []
    print(filters)
    for i, pre in enumerate(pres):
        out = filters[i].transform(pre["rssi"].to_numpy())
        outs.append(out)
    return outs, filters

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
outs, _ = filter_out([pre1], [abs1])

In [9]:
from scipy.signal import medfilt, order_filter

In [14]:
pre1.rssi

0     -29.0
1     -27.0
2     -26.0
3     -27.0
4     -26.0
       ... 
195   -25.0
196   -25.0
197   -23.0
198   -23.0
199   -26.0
Name: rssi, Length: 200, dtype: float64

In [23]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import confusion_matrix

for model in [RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier]:
    forest = model()
    forest.fit(pre1.rssi.to_numpy()[:,np.newaxis], y1)
    predict = forest.predict(pre2.rssi.to_numpy()[:,np.newaxis])
    filtered = medfilt(predict, kernel_size=13)
    print(accuracy_score(y2, filtered))
    print(confusion_matrix(y2, filtered, labels=[0, 1]).ravel())

0.98
[150   0   4  46]
0.98
[150   0   4  46]
0.98
[150   0   4  46]
0.98
[150   0   4  46]


In [ ]:
# axs.append(fig.add_subplot(n, 1, k))
#         axs[k - 1].plot(pre.rssi.to_numpy(), label=legend[0], linestyle="solid")
#         axs[k-1].set_xticks(ticks)
#         axs[k-1].set_xlabel("Время, с")
#         axs[k-1].set_ylabel("RSSI, dBm")
#         axs[k - 1].plot(out, label=legend[1], linestyle="dashed")
#         # axs[k - 1].legend()
#         axs[k - 1].set(title=titles[k-1])
plt.rcParams.update({'font.size': 25})
plt.figure(figsize=(20, 6))
plt.plot(pre1.rssi.to_numpy(), label="RSSI до обработки", linestyle=":", marker=".")
plt.plot(outs[0], label="RSSI после обработки", linestyle="--", marker="s")
plt.title("Замеры RSSI в жилой комнате")
plt.legend(loc="lower right")
plt.savefig("shibaev_1.png")

In [ ]:
plt.rcParams.update({'font.size': 25})
plt.figure(figsize=(20, 6))
plt.plot(pre1.rssi.to_numpy(), label="RSSI до обработки", linestyle=":", marker=".")
plt.plot(outs[0], label="RSSI после обработки", linestyle="--", marker="s")
plt.title("Замеры RSSI в офисе")
plt.legend(loc="lower right")
plt.savefig("shibaev_2.png")